In [144]:
import json
import matplotlib.pyplot as plt
from collections import Counter
import statistics
from datetime import datetime
from collections import defaultdict
from torchtext.vocab import GloVe
from sklearn.feature_extraction.text import CountVectorizer
import re

import torch
from torch import nn
import torch.nn.functional as F
from sklearn.feature_extraction.text import CountVectorizer

from collections import Counter


from sklearn.model_selection import train_test_split



from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import nltk


In [8]:
# business = []
# with open("../dataset/business_all.json", "r") as file:
#     for line in file:
#         business.append(json.loads(line))

# checkin = []
# with open("../dataset/checkin_all.json", "r") as file:
#     for line in file:
#         checkin.append(json.loads(line))

# review = []
# with open("../dataset/review_all.json", "r") as file:
#     for line in file:
#         review.append(json.loads(line))

# user = []
# with open("../dataset/user_all.json", "r") as file:
#     for line in file:
#         user.append(json.loads(line))


review_train = []
with open("../dataset/review_train.json", "r") as file:
    for line in file:
        review_train.append(json.loads(line))

review_test = []
with open("../dataset/review_test.json", "r") as file:
    for line in file:
        review_test.append(json.loads(line))




In [9]:
print(review_train[0]['text'])

Terrible place to shop!!! The manager is an expletive i cannot say on here. I went and bought several nails and screws as well as a rug and other items. While i was checking out the cashier charged me incorrectly for some of the screws, actually charged me more than what they were so i told the cashier and she corrected her mistake. While shopping i was there for about an hour because i was looking at paint, i even talked to a guy while i was there about different paint options as well as another lady about drill bits and about my cat condo project. anyway after i left i realize after i got home the cashier had also forgotten to give me a receipt. the rug i purchased did not work where i wanted it so i attempted to return it about 1 week after i purchased it, i went back the following weekend and the manager refused. i have never heard of refusing to return something or even give a store credit. im not one to return stuff so i was completely shocked. when i put up a stink with the mana

In [177]:
class NNBOW2(nn.Module):
    def __init__(self, input_size, hidden_size):
        super().__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, 1)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = torch.sigmoid(x) * 4 + 1
        return x

class NNBOW3(nn.Module):
    def __init__(self, input_size, hidden_size):
        super().__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.fc3 = nn.Linear(hidden_size, 1)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        x = torch.sigmoid(x) * 4 + 1
        return x

class NNBOW4(nn.Module):
    def __init__(self, input_size, hidden_size):
        super().__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.fc3 = nn.Linear(hidden_size, hidden_size//2)
        self.fc4 = nn.Linear(hidden_size//2, 1)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = self.fc4(x)
        return x

In [145]:
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/outakaratakara/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/outakaratakara/nltk_data...


True

In [146]:
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

In [147]:
def preprocess_review_text(review_text):
    review_text = review_text.lower()
    review_text = re.sub(r'\W+', ' ', review_text)
    words = [word for word in review_text.split() if word not in stop_words]
    processed_words = [lemmatizer.lemmatize(word, pos='v') for word in words]
    return processed_words


In [ ]:
# review_text_train = [
#     re.sub(r'\W+', ' ', review['text'].lower()).split()
#     for review in review_train
# ]

# review_text_test = [
#     re.sub(r'\W+', ' ', review['text'].lower()).split()
#     for review in review_test
# ]

review_text_train = [
    preprocess_review_text(review['text']) for review in review_train
]

review_text_test = [
    preprocess_review_text(review['text']) for review in review_test
]

In [40]:
rating_train = [review['stars'] for review in review_train]
rating_test = [review['stars'] for review in review_test]

In [149]:
vectorizer = CountVectorizer(max_features=512)

review_text_sentences_train = [" ".join(words) for words in review_text_train]
bow_embeddings_train = (vectorizer.fit_transform(review_text_sentences_train)).toarray()

review_text_sentences_test = [" ".join(words) for words in review_text_test]
bow_embeddings_test = (vectorizer.fit_transform(review_text_sentences_test)).toarray()

In [150]:
X_train, X_val, y_train, y_val = train_test_split(
    bow_embeddings_train,  
    rating_train,                 
    test_size=0.2,              
    random_state=42               
)

In [151]:
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
X_val_tensor = torch.tensor(X_val, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32).view(-1, 1)  
y_val_tensor = torch.tensor(y_val, dtype=torch.float32).view(-1, 1)   
X_test_tensor = torch.tensor(bow_embeddings_test, dtype=torch.float32)
y_test_tensor = torch.tensor(rating_test, dtype=torch.float32).view(-1, 1)

In [152]:
input_size = X_train_tensor.shape[1]
hidden_size = 128 

criterion = nn.MSELoss()

In [153]:
def train_model(model, criterion, optimizer, X_train, y_train, X_val, y_val, num_epochs=20):
    for epoch in range(num_epochs):
        model.train()
        predictions = model(X_train)
        loss = criterion(predictions, y_train)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        model.eval()
        with torch.no_grad():
            val_predictions = model(X_val)
            val_loss = criterion(val_predictions, y_val)

        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {loss.item():.4f}, Val Loss: {val_loss.item():.4f}")

In [154]:
def test_model(model, criterion, X_test, y_test):
    model.eval()
    with torch.no_grad():
        test_predictions = model(X_test)
        test_loss = criterion(test_predictions, y_test)
    print(f"Test Loss: {test_loss.item():.4f}")
    return test_predictions

In [174]:
model_2 = NNBOW2(input_size, hidden_size)
optimizer = torch.optim.Adam(model_2.parameters(), lr=0.01)

train_model(
    model=model_2,
    criterion=criterion,
    optimizer=optimizer,
    X_train=X_train_tensor,
    y_train=y_train_tensor,
    X_val=X_val_tensor,
    y_val=y_val_tensor,
    num_epochs=140
)

Epoch 1/140, Loss: 1.9780, Val Loss: 1.7876
Epoch 2/140, Loss: 1.7808, Val Loss: 1.7088
Epoch 3/140, Loss: 1.7024, Val Loss: 1.6513
Epoch 4/140, Loss: 1.6442, Val Loss: 1.5740
Epoch 5/140, Loss: 1.5654, Val Loss: 1.4820
Epoch 6/140, Loss: 1.4711, Val Loss: 1.4019
Epoch 7/140, Loss: 1.3891, Val Loss: 1.3571
Epoch 8/140, Loss: 1.3438, Val Loss: 1.3319
Epoch 9/140, Loss: 1.3182, Val Loss: 1.2993
Epoch 10/140, Loss: 1.2852, Val Loss: 1.2592
Epoch 11/140, Loss: 1.2445, Val Loss: 1.2233
Epoch 12/140, Loss: 1.2082, Val Loss: 1.1989
Epoch 13/140, Loss: 1.1832, Val Loss: 1.1838
Epoch 14/140, Loss: 1.1676, Val Loss: 1.1716
Epoch 15/140, Loss: 1.1549, Val Loss: 1.1576
Epoch 16/140, Loss: 1.1404, Val Loss: 1.1412
Epoch 17/140, Loss: 1.1236, Val Loss: 1.1248
Epoch 18/140, Loss: 1.1068, Val Loss: 1.1111
Epoch 19/140, Loss: 1.0928, Val Loss: 1.1013
Epoch 20/140, Loss: 1.0824, Val Loss: 1.0941
Epoch 21/140, Loss: 1.0749, Val Loss: 1.0879
Epoch 22/140, Loss: 1.0683, Val Loss: 1.0812
Epoch 23/140, Loss:

In [182]:
model_3 = NNBOW3(input_size, hidden_size)

optimizer_3 = torch.optim.Adam(model_3.parameters(), lr=0.01)
train_model(
    model=model_3,
    criterion=criterion,
    optimizer=optimizer_3,
    X_train=X_train_tensor,
    y_train=y_train_tensor,
    X_val=X_val_tensor,
    y_val=y_val_tensor,
    num_epochs=60
)

Epoch 1/60, Loss: 2.0656, Val Loss: 1.8502
Epoch 2/60, Loss: 1.8482, Val Loss: 1.5998
Epoch 3/60, Loss: 1.5936, Val Loss: 1.5339
Epoch 4/60, Loss: 1.5246, Val Loss: 1.3668
Epoch 5/60, Loss: 1.3557, Val Loss: 1.2380
Epoch 6/60, Loss: 1.2252, Val Loss: 1.1568
Epoch 7/60, Loss: 1.1428, Val Loss: 1.0818
Epoch 8/60, Loss: 1.0678, Val Loss: 1.0731
Epoch 9/60, Loss: 1.0578, Val Loss: 1.0593
Epoch 10/60, Loss: 1.0424, Val Loss: 1.0361
Epoch 11/60, Loss: 1.0175, Val Loss: 1.0300
Epoch 12/60, Loss: 1.0098, Val Loss: 1.0042
Epoch 13/60, Loss: 0.9825, Val Loss: 0.9732
Epoch 14/60, Loss: 0.9503, Val Loss: 0.9532
Epoch 15/60, Loss: 0.9300, Val Loss: 0.9220
Epoch 16/60, Loss: 0.8987, Val Loss: 0.9119
Epoch 17/60, Loss: 0.8883, Val Loss: 0.8902
Epoch 18/60, Loss: 0.8667, Val Loss: 0.8865
Epoch 19/60, Loss: 0.8635, Val Loss: 0.8718
Epoch 20/60, Loss: 0.8480, Val Loss: 0.8721
Epoch 21/60, Loss: 0.8468, Val Loss: 0.8598
Epoch 22/60, Loss: 0.8342, Val Loss: 0.8581
Epoch 23/60, Loss: 0.8328, Val Loss: 0.84

In [172]:
test_model(
    model=model_2,
    criterion=criterion,
    X_test=X_test_tensor,
    y_test=y_test_tensor
)

Test Loss: 1.3367


tensor([[3.6611, 3.6580, 3.6729,  ..., 3.6492, 3.6616, 3.6763],
        [4.1086, 4.1287, 4.1296,  ..., 4.1179, 4.1237, 4.1064],
        [3.3301, 3.3489, 3.3594,  ..., 3.3276, 3.3453, 3.3473],
        ...,
        [4.4614, 4.4703, 4.4555,  ..., 4.4748, 4.4638, 4.4493],
        [4.1353, 4.1184, 4.1367,  ..., 4.1181, 4.1234, 4.1167],
        [4.2338, 4.2453, 4.2427,  ..., 4.2356, 4.2406, 4.2351]])

In [181]:
test_model(
    model=model_3,
    criterion=criterion,
    X_test=X_test_tensor,
    y_test=y_test_tensor
)

Test Loss: 1.3502


tensor([[4.0311],
        [3.7657],
        [3.0412],
        ...,
        [3.7832],
        [4.3319],
        [4.1538]])